In [1]:
import numpy as np
from astropy import *
from astropy.table import Table


In [61]:
M = 1.17
P = 628.
year = 365.25
aearth = 1.
rsun = 6.957e8
au = 149597870700 # m

print rsun/au

0.00465046726096


In [62]:
const = aearth**3 / year**2

In [63]:
a = (P**2 * const)**(1/3.)
print a

1.43520171971


In [64]:
R = M**0.8
print R

1.13383194656


In [65]:
def L(M):
    logl = 4.328*np.log(M) -0.002
    return np.exp(logl)
    
print L(M)

1.96897327138


In [74]:
def teffpred(L,R):
    return 5777.*(L/R**2.)**0.25

print teffpred(L(M),R)

6426.69926906


In [75]:
def teq(Teff,R,a,f,A):
    '''Teff is star, R is stellar radius, a is semimajor axis, A is Bond albedo, f is the 
    geometric factor of reradiation'''
    
    return Teff*np.sqrt(R*rsun/2./(a*au))*(f*(1.-A))**0.25

In [76]:
tstar = teffpred(L(M),R)

In [81]:
tplan = teq(tstar,R,a,2.0,0.5)
print tplan

275.447698188


In [82]:
L(M)/(a**2)

0.95590322869547595

In [83]:
R

1.133831946557216

In [84]:
## earth

Tearth = teq(5777,1,1,2.0,0.5)
print Tearth

278.570847443


In [87]:
mas2rad = 1000*3600*180./np.pi

In [94]:
## today

giant_teff = 3934.

distance = 19.34 # pc
angular = 20.58e-3
print angular

linear = angular*distance*(au)/rsun



0.02058


In [95]:
tgiant = teq(giant_teff,linear,a,2.0,0.5)

In [96]:
tgiant

1464.9196508550642